# 择时策略
什么是择时策略？
> 根据价格走势，选择时机买入卖出

eg: 通过一个简单的案例策略，来演示择时策略

## 步骤：
### 一、 导入数据


In [1]:
from util import location

ts_code = "000002.SZ"
root = "../data_store/"
df = location.read_from_csv(ts_code=ts_code,root=root)
df

,ts_code,trade_date,open,high,low,close,pre_close,vol,amount
0,000002.SZ,1991-01-29,14.58,14.58,14.58,14.58,1.00,3.00,22.000
1,000002.SZ,1991-01-30,14.51,14.51,14.51,14.51,14.58,17.00,123.000
2,000002.SZ,1991-02-04,14.66,14.66,14.66,14.66,14.59,56.00,410.000
3,000002.SZ,1991-02-05,14.73,14.73,14.73,14.73,14.66,29.00,213.000
4,000002.SZ,1991-02-06,14.80,14.80,14.80,14.80,14.73,29.00,215.000
...,...,...,...,...,...,...,...,...,...
7176,000002.SZ,2021-07-13,23.84,24.14,23.73,23.84,23.82,429413.08,1026027.218
7177,000002.SZ,2021-07-14,23.81,23.88,23.20,23.21,23.84,748482.33,1753035.681
7178,000002.SZ,2021-07-15,23.21,23.73,23.01,23.52,23.21,639812.43,1498254.942
7179,000002.SZ,2021-07-16,23.40,24.05,23.35,23.95,23.52,643153.72,1534958.952


### 二、计算复权价

In [2]:
from acquire import stock

df = stock.Indicators(df=df).adj_price
df

,ts_code,trade_date,open,high,low,close,pre_close,vol,amount,pct_change,adj_factor,hfq_close,qfq_close,qfq_open,qfq_high,qfq_low,hfq_open,hfq_high,hfq_low
0,000002.SZ,1991-01-29,14.58,14.58,14.58,14.58,1.00,3.00,22.000,13.580000,14.580000,14.580000,0.094736,0.094736,0.094736,0.094736,14.580000,14.580000,14.580000
1,000002.SZ,1991-01-30,14.51,14.51,14.51,14.51,14.58,17.00,123.000,-0.004801,14.510000,14.510000,0.094282,0.094282,0.094282,0.094282,14.510000,14.510000,14.510000
2,000002.SZ,1991-02-04,14.66,14.66,14.66,14.66,14.59,56.00,410.000,0.004798,14.579616,14.579616,0.094734,0.094734,0.094734,0.094734,14.579616,14.579616,14.579616
3,000002.SZ,1991-02-05,14.73,14.73,14.73,14.73,14.66,29.00,213.000,0.004775,14.649232,14.649232,0.095186,0.095186,0.095186,0.095186,14.649232,14.649232,14.649232
4,000002.SZ,1991-02-06,14.80,14.80,14.80,14.80,14.73,29.00,215.000,0.004752,14.718849,14.718849,0.095639,0.095639,0.095639,0.095639,14.718849,14.718849,14.718849
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7176,000002.SZ,2021-07-13,23.84,24.14,23.73,23.84,23.82,429413.08,1026027.218,0.000840,3668.992104,3668.992104,23.840000,23.840000,23.840000,23.840000,3668.992104,3668.992104,3668.992104
7177,000002.SZ,2021-07-14,23.81,23.88,23.20,23.21,23.84,748482.33,1753035.681,-0.026426,3572.034679,3572.034679,23.210000,23.810000,23.210000,23.210000,3664.375084,3572.034679,3572.034679
7178,000002.SZ,2021-07-15,23.21,23.73,23.01,23.52,23.21,639812.43,1498254.942,0.013356,3619.743888,3619.743888,23.520000,23.210000,23.520000,23.520000,3572.034679,3619.743888,3619.743888
7179,000002.SZ,2021-07-16,23.40,24.05,23.35,23.95,23.52,643153.72,1534958.952,0.018282,3685.921178,3685.921178,23.950000,23.400000,23.950000,23.950000,3601.275807,3685.921178,3685.921178


### 三、计算涨跌停价格
1. 获取股票历史名称（因为st股的涨跌幅是不同的，科创板涨跌幅也是不同的）
2. 对涨跌停特征列的每一行用函数：
```
from decimal import Decimal, ROUND_HALF_UP
.apply(lambda x: float(Decimal(x * 100).quantize(Decimal('1'), rounding=ROUND_HALF_UP) / 100))
```

### 四、选择策略进行回测分析
- 我们这里使用移动平均线的算法作为举例